# Data Generation

## Import Packages

In [6]:
import numpy as np
import scipy
from scipy.stats import norm
import pandapower as pp
import pandas as pd
import pandapower.networks
import tensorflow as tf
import csv
import torch

## Input Data

In [7]:
case = pp.networks.case9()
number_of_bus = case.bus.shape[0]
baseMVA = 100

number_of_data = 2500       # Including data for both Training and Testing
number_of_training_data = 2000
number_of_test_data = np.subtract(number_of_data,number_of_training_data)

load = case.load.iloc[:,6:8]
load_bus_number = case.load.iloc[:,0]

Pgenerators = np.zeros((number_of_bus,1))
Pgenerators[case.gen.iloc[:,0],0] = -case.gen.iloc[:,4]

PQload = np.zeros((number_of_bus,2))
PQload[case.load.iloc[:,0],:] = case.load.iloc[:,6:8]



### NB FB TB R X PL QL PG QG V Delta

Data_form = np.hstack(((np.array(case.bus_geodata.iloc[:,0:2])),\
                       PQload,Pgenerators))

## Roulette Wheel Algorithm Using Normal-PDF to Generate Scenarios

In [8]:
Scenario = np.zeros([number_of_data,Data_form.shape[0],Data_form.shape[1]])

for i in range(number_of_data):
    
    Scenario[i,:,:] = Data_form
    Scenario[i,load_bus_number,2:4] = norm.ppf(np.random.rand(load.shape[0],1),\
                                                              load,np.multiply(0.1,load))

Training_Data = Scenario[0:number_of_training_data,:,:]
Test_Data = Scenario[number_of_training_data:,:,:]




## Implement Power Flow to Get Labels

In [9]:
Label = np.zeros((number_of_data,number_of_bus,2))

for i in range(number_of_data):
    
    case.load.iloc[:,6:8] = Scenario[i,case.load.iloc[:,0],2:4]
    
    pp.runpp(case, algorithm='nr', calculate_voltage_angles='auto', init='auto',\
                  max_iteration='auto', tolerance_mva=1e-08, trafo_model='t',\
                  trafo_loading='current', enforce_q_lims=False,\
                  check_connectivity=True, voltage_depend_loads=True,\
                 consider_line_temperature=False)
    
    Label[i,:,:] = case.res_bus.iloc[:,0:2]
    
Training_Label = Label[0:number_of_training_data,:,:]
Test_Label = Label[number_of_training_data:,:,:]


## Export Data

In [130]:
Final_Training_Data = pd.DataFrame(Training_Data.reshape(number_of_training_data,-1))
Final_Training_Label  = pd.DataFrame(Training_Label.reshape(number_of_training_data,-1))
Final_Training_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_data.csv",index = False)
Final_Training_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_label.csv",index = False)

Final_Test_Data   = pd.DataFrame(Test_Data.reshape(number_of_test_data,-1))
Final_Test_Label  = pd.DataFrame(Test_Label.reshape(number_of_test_data,-1))
Final_Test_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_data.csv",index = False)
Final_Test_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_label.csv",index = False)




In [10]:
case

This pandapower network includes the following parameter tables:
   - bus (9 elements)
   - load (3 elements)
   - gen (2 elements)
   - ext_grid (1 element)
   - line (9 elements)
   - poly_cost (3 elements)
   - bus_geodata (9 elements)
 and the following results tables:
   - res_bus (9 elements)
   - res_line (9 elements)
   - res_ext_grid (1 element)
   - res_load (3 elements)
   - res_gen (2 elements)

In [17]:
case.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-63.941830,-20.699227
1,1.000000,9.865960,-163.000000,-14.382162
2,1.000000,5.132262,-85.000000,5.029489
3,0.988763,-2.134707,0.000000,0.000000
4,0.981167,-3.319419,78.590232,26.196744
5,1.004183,2.289082,0.000000,0.000000
6,0.985085,0.787180,104.290192,36.501567
7,0.996234,3.996625,0.000000,0.000000
8,0.959163,-4.076505,124.373012,49.749205


In [16]:
case.gen

,bus,controllable,in_service,name,p_mw,scaling,sn_mva,type,vm_pu,slack,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar,slack_weight
0,1,True,True,None,163.0,1.0,NaN,None,1.0,False,300.0,10.0,300.0,-300.0,0.0
1,2,True,True,None,85.0,1.0,NaN,None,1.0,False,270.0,10.0,300.0,-300.0,0.0
